### Assignment3- PartC
Use Keras-NLP for these nlp tasks -

- Inference with a pretrained classifier

- Fine tuning a pretrained backbone

- Fine tuning with user-controlled preprocessing

- Fine tuning a custom modes

### Inference with a pretrained classifier


In [1]:
!pip install keras-nlp


In [2]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.5 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# New example review
review = "The book was an uninspiring read, lacking depth in characters and plot."

# Encode the review and prepare for the model (add batch dimension)
inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)

# Predict sentiment
with torch.no_grad():
    logits = model(**inputs).logits

# Convert logits to probabilities
probs = softmax(logits, dim=1)

# Interpret the prediction
# The output probabilities are for [negative, positive] classes
print(probs)  # This will show you the probabilities for each class

# Determining the predicted class based on higher probability
predicted_class = "positive" if probs[0][1] > probs[0][0] else "negative"
print(f"Predicted sentiment: {predicted_class}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tensor([[9.9980e-01, 2.0041e-04]])
Predicted sentiment: negative


## Fine-Tuning a Pretrained Backbone

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import keras_nlp

In [3]:
import pandas as pd

# Load the Parquet file
df = pd.read_csv('/content/amazon_alexa.csv')

# Display the first few rows of the dataframe to understand its structure
print(df.head())


   rating       date         variation  \
0       5  31-Jul-18  Charcoal Fabric    
1       5  31-Jul-18  Charcoal Fabric    
2       4  31-Jul-18    Walnut Finish    
3       5  31-Jul-18  Charcoal Fabric    
4       5  31-Jul-18  Charcoal Fabric    

                                    verified_reviews  feedback  
0                                      Love my Echo!         1  
1                                          Loved it!         1  
2  Sometimes while playing a game, you can answer...         1  
3  I have had a lot of fun with this thing. My 4 ...         1  
4                                              Music         1  


In [8]:
import tensorflow as tf
from tensorflow.keras.utils import text_dataset_from_directory
import keras_nlp

# This assumes you have already loaded your dataframe 'df' from the Parquet file.
sentences = df['verified_reviews'].tolist()
labels = df['feedback'].tolist()

# Create a dataset from the sentences and labels
dataset = tf.data.Dataset.from_tensor_slices((sentences, labels))

# Batch the dataset
batch_size = 16
dataset = dataset.batch(batch_size)

# Split the dataset into training and validation
# Here, I'm just showing a simple split, but you should use a proper method to split your dataset.
train_size = int(0.8 * len(sentences))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Initialize the BertClassifier
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
# classifier = keras_nlp.models.BertClassifier.from_preset(
#     "bert_tiny_en_uncased",
#     preprocessor=None,
# )
classifier.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1,
)
# Compile the model
# classifier.compile(
#     optimizer='adam',
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy'],
# )

# # Fit the model
# classifier.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=1,
# )


197/197 ━━━━━━━━━━━━━━━━━━━━ 123s 115ms/step - loss: 0.4314 - sparse_categorical_accuracy: 0.8700 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0000e+00


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


#Fine tuning with user-controlled preprocessing


In [9]:
import tensorflow as tf
import keras_nlp
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Let's assume df is your loaded DataFrame from the Parquet file

# Convert DataFrame columns to a list of dictionaries (one per example)
# and split the data into a training set and a validation set
train_sentences = df['verified_reviews'].tolist()
train_labels = df['feedback'].tolist()

# Create TensorFlow datasets from the sentences and labels
train_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels))

# Set your batch size
batch_size = 16

# Shuffle and batch the datasets
train_data = train_data.shuffle(len(train_sentences)).batch(batch_size)

# Initialize the BertPreprocessor
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=128  # Adjust sequence_length to your needs
)

# Apply the preprocessor to the dataset
train_data = train_data.map(lambda x, y: (preprocessor(x), y),
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Cache and prefetch for performance optimization
train_data = train_data.cache().prefetch(tf.data.experimental.AUTOTUNE)

# Split your dataset into training and validation
# Here we'll use 80% for training and 20% for validation
train_size = int(0.8 * len(train_sentences))
val_size = len(train_sentences) - train_size

train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)

# Initialize the BertClassifier without passing a preprocessor, as we have done it already
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
    preprocessor=None,  # Preprocessing has been handled manually
)

# Compile the model
classifier.compile(
    optimizer=Adam(learning_rate=3e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Fit the model on the train dataset and validate on the validation dataset
classifier.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1
)


197/197 ━━━━━━━━━━━━━━━━━━━━ 77s 160ms/step - accuracy: 0.7275 - loss: 0.5593 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


# Fine tuning a custom modes  

In [10]:
import tensorflow as tf
import keras_nlp
import keras
from keras_nlp.models import BertBackbone
from keras_nlp.layers import TransformerEncoder
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

# Let's assume df is your loaded DataFrame from the Parquet file

# Convert DataFrame columns to a list of dictionaries (one per example)
train_sentences = df['verified_reviews'].tolist()
train_labels = df['feedback'].tolist()

# Create TensorFlow datasets from the sentences and labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels))

# Set your batch size
batch_size = 16

# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(len(train_sentences)).batch(batch_size)

# Initialize the BertPreprocessor
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")

# Apply the preprocessor to the dataset
train_dataset = train_dataset.map(lambda x, y: (preprocessor(x), y),
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Cache and prefetch for performance optimization
train_dataset = train_dataset.cache().prefetch(tf.data.experimental.AUTOTUNE)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_sentences))
train_dataset = train_dataset.take(train_size)
val_dataset = train_dataset.skip(train_size)

# Initialize the BertBackbone
backbone = BertBackbone.from_preset("bert_tiny_en_uncased")

# Make the backbone non-trainable
backbone.trainable = False

# Define custom architecture
inputs = backbone.input
sequence = backbone(inputs)["sequence_output"]
for _ in range(2):
    sequence = TransformerEncoder(
        num_heads=2,
        intermediate_dim=512,
        dropout=0.1,
    )(sequence)

# Use [CLS] token output to classify
outputs = keras.layers.Dense(2)(sequence[:, backbone.cls_token_index, :])

# Create the model
model = Model(inputs, outputs)

# Compile the model
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=AdamW(learning_rate=5e-5),
    metrics=[SparseCategoricalAccuracy()],
    jit_compile=True,
)

# Fit the model on the train dataset and validate on the validation dataset
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1,
)

# Show the model summary
model.summary()


197/197 ━━━━━━━━━━━━━━━━━━━━ 84s 179ms/step - loss: 0.4634 - sparse_categorical_accuracy: 0.8294 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0000e+00


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 128), (None,   │      4,385,920 │ padding_mask[0][0],    │
│ (BertBackbone)            │ None, 128)]            │                │ segment_ids[0][0],     │
│                           │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 128)      │        198,272 │ bert_backbone[0][1]    │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_1     │ (None, None, 128)      │        198,272 │ transformer_encoder[0… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_8 (GetItem)      │ (None, 128)            │              0 │ transformer_encoder_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2)              │            258 │ get_item_8[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,576,328 (21.27 MB)

 Trainable params: 396,802 (1.51 MB)

 Non-trainable params: 4,385,920 (16.73 MB)

 Optimizer params: 793,606 (3.03 MB)